# Extracting and Uploading MaStR - Winddata with open-mastr

In [4]:
from open_mastr import Mastr
import pandas as pd
import geopandas as gpd
import supabase_py

In [5]:
tables_list = ["wind", "location", "permit"]
db = Mastr()
#db.download(method = "bulk",
#            data=tables_list)

Data will be written to the following database: sqlite:////home/gunnar/.open-MaStR/data/sqlite/open-mastr.db
If you run into problems, try to delete the database and update the package by running 'pip install --upgrade open-mastr'



In [6]:
### Inspect db
conn = db.engine # Connection engine
tables = pd.read_sql_query('SELECT name from sqlite_master where type= "table";', conn)
df_wind = pd.read_sql_table("wind_extended", conn)
columns_wind = list(df_wind.columns)

In [7]:
### Subset to Land
# df_wind = df_wind[df_wind.Lage == "Windkraft an Land"]
df_wind.Kuestenentfernung.describe() 
df_wind.Kuestenentfernung.unique()  # Only for Offshore
### Remove columns
print(columns_wind)

['NameWindpark', 'Lage', 'Seelage', 'ClusterOstsee', 'ClusterNordsee', 'Hersteller', 'HerstellerId', 'Technologie', 'Typenbezeichnung', 'Nabenhoehe', 'Rotordurchmesser', 'Rotorblattenteisungssystem', 'AuflageAbschaltungLeistungsbegrenzung', 'AuflagenAbschaltungSchallimmissionsschutzNachts', 'AuflagenAbschaltungSchallimmissionsschutzTagsueber', 'AuflagenAbschaltungSchattenwurf', 'AuflagenAbschaltungTierschutz', 'AuflagenAbschaltungEiswurf', 'AuflagenAbschaltungSonstige', 'Wassertiefe', 'Kuestenentfernung', 'Buergerenergie', 'Nachtkennzeichen', 'EegMastrNummer', 'NetzbetreiberMastrNummer', 'Registrierungsdatum', 'EinheitMastrNummer', 'DatumLetzteAktualisierung', 'LokationMastrNummer', 'NetzbetreiberpruefungStatus', 'NetzbetreiberpruefungDatum', 'AnlagenbetreiberMastrNummer', 'Land', 'Bundesland', 'Landkreis', 'Gemeinde', 'Gemeindeschluessel', 'Postleitzahl', 'Gemarkung', 'FlurFlurstuecknummern', 'Strasse', 'StrasseNichtGefunden', 'Hausnummer', 'HausnummerNichtGefunden', 'Adresszusatz', '

In [8]:
### Very few plants actually have an address -> Not useful for displaying?
df_address = df_wind[["Strasse", "StrasseNichtGefunden",
       "Hausnummer", "HausnummerNichtGefunden", "Adresszusatz"]]
df_address.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35227 entries, 0 to 35226
Data columns (total 5 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   Strasse                  4653 non-null   object
 1   StrasseNichtGefunden     34280 non-null  object
 2   Hausnummer               2791 non-null   object
 3   HausnummerNichtGefunden  34280 non-null  object
 4   Adresszusatz             400 non-null    object
dtypes: object(5)
memory usage: 1.3+ MB


In [9]:
### iNSPECT GEO-columns
geo_columns = [ 'Laengengrad',
 'Breitengrad',
 'UtmZonenwert',
 'UtmEast',
 'UtmNorth',
 'GaussKruegerHoch',
 'GaussKruegerRechts']
df_geo = df_wind[geo_columns]
df_geo.info()
df_geo.UtmZonenwert.unique()
### open-mastr has no coordinates beside wgs84

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35227 entries, 0 to 35226
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Laengengrad         34280 non-null  float64
 1   Breitengrad         34280 non-null  float64
 2   UtmZonenwert        0 non-null      object 
 3   UtmEast             0 non-null      float64
 4   UtmNorth            0 non-null      float64
 5   GaussKruegerHoch    0 non-null      float64
 6   GaussKruegerRechts  0 non-null      float64
dtypes: float64(6), object(1)
memory usage: 1.9+ MB


array([None], dtype=object)

In [10]:
columns_remove = ["Lage",	"Seelage",	"ClusterOstsee", "ClusterNordsee", "Strasse", "StrasseNichtGefunden",
       "Hausnummer", "HausnummerNichtGefunden", "Adresszusatz", "NetzbetreiberpruefungStatus", 
       "NetzbetreiberpruefungDatum", "Wassertiefe", "Kuestenentfernung", "UtmZonenwert","UtmEast", "UtmNorth",
 "GaussKruegerHoch", "GaussKruegerRechts"]
columns_wind = [col for col in columns_wind if col not in columns_remove]
df_wind = df_wind[columns_wind]
columns_wind = df_wind.columns

In [20]:
### SQL Create Statement

types_wind = [df_wind[col].dtypes for col in columns_wind]
set(types_wind)

# dictionary mapping the data types: Postgres Data type = pandas data type
data_pgsql_pd = {'bool'= 'bool', 
                 'float8' = 'float64', 
                 'date' = '<M8[ns]', 
                 'varchar'='0'}

for col in columns_wind:
    # SQL create column statement for this column: 'column_name pgsql-type,'. Leave out constraints for now

# leave out geo-columns -> added later on
# add primary key
    

{dtype('bool'), dtype('float64'), dtype('<M8[ns]'), dtype('O')}